In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.tree import DecisionTreeRegressor
from ngboost import NGBRegressor
from ngboost.distns import Normal
from ngboost.scores import CRPScore, MLE
from ngboost.ngboost import NGBoost
from sklearn.tree import DecisionTreeRegressor

from hyperopt import hp, tpe, space_eval
from hyperopt.pyll.base import scope
from hyperopt.fmin import fmin
from hyperopt import STATUS_OK, Trials

import logging

In [2]:
# options
pd.set_option('max_columns',100)
plt.style.use('fivethirtyeight')
warnings.simplefilter('ignore')
seed = 1

In [3]:
# Data dirctory
data_dir = Path('../data/')
data_file = data_dir / 'data.csv'

In [4]:
# Data
# 0:AAR / 1:EAD / 2:ADR / 3:EDD는 고정  , 나머지는 순서 상관 없음
Data = pd.read_csv(data_file, index_col=0)

***
***
***

# Data Selection

In [5]:
# 필요없는 것을 버리기
Data_temp = Data.drop('TMP', axis=1)
Data_temp = Data_temp.drop('TD', axis=1)
Data_temp = Data_temp.drop('HM', axis=1)
Data_temp = Data_temp.drop('PS', axis=1)
Data_temp = Data_temp.drop('PA', axis=1)

#고층바람 너무 높은 고도는 뺴자 
Data_temp = Data_temp.drop('WD_400', axis=1)
Data_temp = Data_temp.drop('WD_500', axis=1)
Data_temp = Data_temp.drop('WD_700', axis=1)
Data_temp = Data_temp.drop('WS_400', axis=1)
Data_temp = Data_temp.drop('WS_500', axis=1)
Data_temp = Data_temp.drop('WS_700', axis=1)

# drop TAF
for i in range(6,30,6):
    Data_temp = Data_temp.drop(f'WDIR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'WSPD_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'WG_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'VIS_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'WC_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'CLA_1LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'BASE_1LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'CLA_2LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'BASE_2LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'CLA_3LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'BASE_3LYR_t{i}', axis=1)

In [6]:
# 각 시간에 맞는 TAF로 나누기
taf6 = [12,18,24]
taf12 = [6,18,24]
taf18 = [6,12,24]
taf24 = [6,12,18]
    
# 각 시간에 맞는 taf 넣기
data_taf = {}
for i in range(6,30,6):
    data_taf[f'Data_{i}'] = Data_temp    
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'WDIR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'WSPD_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'WG_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'VIS_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'WC_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'CLA_1LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'BASE_1LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'CLA_2LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'BASE_2LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'CLA_3LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'BASE_3LYR_t{i}'])
    
Data_6 = data_taf['Data_6']
Data_12 = data_taf['Data_12']
Data_18 = data_taf['Data_18']
Data_24 = data_taf['Data_24']

In [7]:
# 예측할 시간에 맞는 Data로 넣기
# 0-6 : Data_6 / 6-12 : Data_12 / 12-18 : Data_18 / 18-24 : Data_24
Data_raw = Data_6
Data_m = Data_6
Data_m = Data_m.drop('AAR', axis=1)
Data_m = Data_m.drop('ADR', axis=1)


# Arrival
y_a = Data_raw.AAR.to_numpy()
X_a = Data_m.to_numpy()
X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(X_a, y_a, test_size = 0.1, random_state = seed)
X_train_a, X_val_a, y_train_a, y_val_a = train_test_split(X_train_a, y_train_a, test_size=0.11, random_state = 13) 


# Departure
y_d = Data_raw.ADR.to_numpy()
X_d = Data_m.to_numpy()
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_d, y_d, test_size = 0.1, random_state = seed)
X_train_d, X_val_d, y_train_d, y_val_d = train_test_split(X_train_d, y_train_d, test_size=0.11, random_state = 13) 

# val은 hyperparameter 검증에 사용
# 0.11 x 0.9 = 0.099

print('Training Data shape : ', Data_m.shape)
Data_m.head()

Training Data shape :  (17520, 46)


,EAD,EDD,year,month,day,hour,DayName,Arpt_cond,P_Airp,P_AAR,P_ADR,Arrival_remainder,Departure_remainder,WD_850,WD_925,WD_1000,WS_850,WS_925,WS_1000,WD,WSPD,WS_GST,VIS,WC,RN,CA_TOT,CLA_1LYR,BASE_1LYR,CLA_2LYR,BASE_2LYR,CLA_3LYR,BASE_3LYR,CLA_4LYR,BASE_4LYR,RVR,WDIR_t6,WSPD_t6,WG_t6,VIS_t6,WC_t6,CLA_1LYR_t6,BASE_1LYR_t6,CLA_2LYR_t6,BASE_2LYR_t6,CLA_3LYR_t6,BASE_3LYR_t6
0,2.0,11.0,2018,1,1,0,1,1.0,1.0,0.0,0.0,0.0,0.0,320.0,325.0,245.0,28.0,19.0,3.0,34,5,0.0,1000,1,0.0,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
1,1.0,5.0,2018,1,1,1,1,1.0,1.0,3.0,11.0,0.0,0.0,320.0,325.0,245.0,28.0,19.0,3.0,3,1,0.0,1000,1,0.0,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
2,1.0,4.0,2018,1,1,2,1,1.0,1.0,0.0,4.0,1.0,1.0,320.0,325.0,245.0,28.0,19.0,3.0,35,1,0.0,1000,1,0.0,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
3,0.0,1.0,2018,1,1,3,1,1.0,1.0,0.0,4.0,1.0,0.0,320.0,325.0,245.0,28.0,19.0,3.0,0,0,0.0,1000,1,0.0,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
4,16.0,1.0,2018,1,1,4,1,1.0,1.0,2.0,1.0,0.0,0.0,320.0,325.0,245.0,28.0,19.0,3.0,19,1,0.0,1000,1,0.0,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0


***
***
***

# NGBoost

## Hypteropt

In [8]:
logging.basicConfig(
    # filename = "errors.log",
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger()

In [9]:
b1 = DecisionTreeRegressor(criterion = "friedman_mse",                 # “mse”, “friedman_mse”, “mae”, “poisson”
                                     min_samples_split = 3,             # The minimum number of samples required to split an internal node
                                     min_samples_leaf = 1,              # The minimum number of samples required to be at a leaf node
                                     min_weight_fraction_leaf = 0.0,    # The minimum weighted fraction of the sum total of weights required to be at a leaf node
                                     max_depth = None,                  # The maximum depth of the tree
                                     max_leaf_nodes = 127,              # Grow a tree with 'max_leaf_nodes' in best-first fashion
                                     splitter = "best",                 # The strategy used to choose the split at each node
                                     random_state = seed)
b2 = DecisionTreeRegressor(criterion = "friedman_mse",                 # “mse”, “friedman_mse”, “mae”, “poisson”
                                     min_samples_split = 3,             # The minimum number of samples required to split an internal node
                                     min_samples_leaf = 1,              # The minimum number of samples required to be at a leaf node
                                     min_weight_fraction_leaf = 0.0,    # The minimum weighted fraction of the sum total of weights required to be at a leaf node
                                     max_depth = None,                  # The maximum depth of the tree
                                     max_leaf_nodes = 255,              # Grow a tree with 'max_leaf_nodes' in best-first fashion
                                     splitter = "best",                 # The strategy used to choose the split at each node
                                     random_state = seed)
b3 = DecisionTreeRegressor(criterion = "friedman_mse",                 # “mse”, “friedman_mse”, “mae”, “poisson”
                                     min_samples_split = 3,             # The minimum number of samples required to split an internal node
                                     min_samples_leaf = 1,              # The minimum number of samples required to be at a leaf node
                                     min_weight_fraction_leaf = 0.0,    # The minimum weighted fraction of the sum total of weights required to be at a leaf node
                                     max_depth = None,                  # The maximum depth of the tree
                                     max_leaf_nodes = 63,              # Grow a tree with 'max_leaf_nodes' in best-first fashion
                                     splitter = "best",                 # The strategy used to choose the split at each node
                                     random_state = seed)


space = {
    'learning_rate':hp.quniform('learning_rate', .0005, 0.01, 0.001),
    'minibatch_frac':hp.quniform('minibatch_frac', 0.5, 1.0, 0.1),
    'col_sample':hp.quniform('col_sample', 0.5, 1.0, 0.1),
    'Base':hp.choice('Base', [b1, b2, b3])
}

default_params = {"n_estimators":10000000,
                  "verbose_eval":100,
                  "random_state":13}

In [27]:
def objective_a(params):
    
    params.update(default_params)
    
    print(params)
    ngb_arrival = NGBRegressor(**params).fit(X_train_a, y_train_a,
                                             X_val = X_val_a, 
                                             Y_val = y_val_a, 
                                             early_stopping_rounds = 10)
    loss = ngb_arrival.evals_result['val']['LOGSCORE'][ngb_arrival.best_val_loss_itr]
    logger.info(params)
    results = {'loss':loss, 'status':STATUS_OK}
    
    return results

In [28]:
def objective_d(params):
    
    params.update(default_params)
    
    print(params)
    ngb_departure = NGBRegressor(**params).fit(X_train_d, y_train_d,
                                             X_val = X_val_d, 
                                             Y_val = y_val_d, 
                                             early_stopping_rounds = 10)
    loss = ngb_departure.evals_result['val']['LOGSCORE'][ngb_departure.best_val_loss_itr]
    logger.info(params)
    results = {'loss':loss, 'status':STATUS_OK}
    
    return results

In [29]:
# Arrival 

TRIALS = Trials()
logger.info("Start parameter optimization...")


with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    best = fmin(fn=objective_a,
                space=space,
                algo=tpe.suggest,
                max_evals=100,
                trials=TRIALS)
logger.info("...done")

2021-07-27 23:25:52 INFO     Start parameter optimization...


  0%|                                                                                                               | 0/100 [00:00<?, ?trial/s, best loss=?]

2021-07-27 23:25:52 INFO     build_posterior_wrapper took 0.001559 seconds
2021-07-27 23:25:52 INFO     TPE using 0 trials


{'Base': DecisionTreeRegressor(criterion='friedman_mse', max_leaf_nodes=63,                                                                                 
                      min_samples_split=3, random_state=1), 'col_sample': 0.6000000000000001, 'learning_rate': 0.009000000000000001, 'minibatch_frac': 0.5, 'n_estimators': 10000000, 'verbose_eval': 100, 'random_state': 13}
[iter 0] loss=3.8284 val_loss=3.8174 scale=2.0000 norm=18.0475                                                                                              
[iter 100] loss=2.9119 val_loss=2.9209 scale=2.0000 norm=5.7630                                                                                             
[iter 200] loss=2.4331 val_loss=2.5089 scale=2.0000 norm=4.1815                                                                                             
[iter 300] loss=2.2007 val_loss=2.4028 scale=2.0000 norm=3.8838                                                                                      

2021-07-27 23:26:17 INFO     {'Base': DecisionTreeRegressor(criterion='friedman_mse', max_leaf_nodes=63,
                      min_samples_split=3, random_state=1), 'col_sample': 0.6000000000000001, 'learning_rate': 0.009000000000000001, 'minibatch_frac': 0.5, 'n_estimators': 10000000, 'verbose_eval': 100, 'random_state': 13}


...

2021-07-28 02:44:57 INFO     build_posterior_wrapper took 0.003000 seconds
2021-07-28 02:44:57 INFO     TPE using 99/99 trials with best loss 2.381752


{'Base': DecisionTreeRegressor(criterion='friedman_mse', max_leaf_nodes=255,                                                                                
                      min_samples_split=3, random_state=1), 'col_sample': 0.7000000000000001, 'learning_rate': 0.006, 'minibatch_frac': 0.7000000000000001, 'n_estimators': 10000000, 'verbose_eval': 100, 'random_state': 13}
[iter 0] loss=3.8314 val_loss=3.8264 scale=2.0000 norm=18.1206                                                                                              
[iter 100] loss=3.0915 val_loss=3.1145 scale=2.0000 norm=6.8758                                                                                             
[iter 200] loss=2.6023 val_loss=2.6967 scale=2.0000 norm=3.9166                                                                                             
[iter 300] loss=2.2006 val_loss=2.4771 scale=2.0000 norm=3.0821                                                                                      

2021-07-28 02:45:48 INFO     {'Base': DecisionTreeRegressor(criterion='friedman_mse', max_leaf_nodes=255,
                      min_samples_split=3, random_state=1), 'col_sample': 0.7000000000000001, 'learning_rate': 0.006, 'minibatch_frac': 0.7000000000000001, 'n_estimators': 10000000, 'verbose_eval': 100, 'random_state': 13}


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [3:19:56<00:00, 119.96s/trial, best loss: 2.381752374746717]

2021-07-28 02:45:48 INFO     ...done


In [30]:
best_params = space_eval(space, best)
best_params

{'Base': DecisionTreeRegressor(criterion='friedman_mse', max_leaf_nodes=63,
                       min_samples_split=3, random_state=1),
 'col_sample': 0.7000000000000001,
 'learning_rate': 0.009000000000000001,
 'minibatch_frac': 0.5}

In [32]:
# Departure 

TRIALS = Trials()
logger.info("Start parameter optimization...")


with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    best = fmin(fn=objective_a,
                space=space,
                algo=tpe.suggest,
                max_evals=100,
                trials=TRIALS)
logger.info("...done")

2021-07-28 07:37:27 INFO     Start parameter optimization...


  0%|                                                                                                               | 0/100 [00:00<?, ?trial/s, best loss=?]

2021-07-28 07:37:27 INFO     build_posterior_wrapper took 0.001003 seconds
2021-07-28 07:37:27 INFO     TPE using 0 trials


{'Base': DecisionTreeRegressor(criterion='friedman_mse', max_leaf_nodes=255,                                                                                
                      min_samples_split=3, random_state=1), 'col_sample': 0.6000000000000001, 'learning_rate': 0.009000000000000001, 'minibatch_frac': 0.6000000000000001, 'n_estimators': 10000000, 'verbose_eval': 100, 'random_state': 13}
[iter 0] loss=3.8305 val_loss=3.8172 scale=2.0000 norm=18.0903                                                                                              
[iter 100] loss=2.8622 val_loss=2.9054 scale=2.0000 norm=5.1573                                                                                             
[iter 200] loss=2.2288 val_loss=2.4882 scale=2.0000 norm=3.1797                                                                                             
== Early stopping achieved.                                                                                                           

2021-07-28 07:37:54 INFO     {'Base': DecisionTreeRegressor(criterion='friedman_mse', max_leaf_nodes=255,
                      min_samples_split=3, random_state=1), 'col_sample': 0.6000000000000001, 'learning_rate': 0.009000000000000001, 'minibatch_frac': 0.6000000000000001, 'n_estimators': 10000000, 'verbose_eval': 100, 'random_state': 13}


...

2021-07-28 11:39:02 INFO     build_posterior_wrapper took 0.002924 seconds
2021-07-28 11:39:02 INFO     TPE using 99/99 trials with best loss 2.380576


{'Base': DecisionTreeRegressor(criterion='friedman_mse', max_leaf_nodes=127,                                                                                
                      min_samples_split=3, random_state=1), 'col_sample': 0.5, 'learning_rate': 0.007, 'minibatch_frac': 0.7000000000000001, 'n_estimators': 10000000, 'verbose_eval': 100, 'random_state': 13}
[iter 0] loss=3.8314 val_loss=3.8233 scale=2.0000 norm=18.1206                                                                                              
[iter 100] loss=3.0576 val_loss=3.0725 scale=2.0000 norm=6.7581                                                                                             
[iter 200] loss=2.5686 val_loss=2.6427 scale=2.0000 norm=4.2353                                                                                             
[iter 300] loss=2.2379 val_loss=2.4518 scale=2.0000 norm=3.6266                                                                                             
== Earl

2021-07-28 11:39:44 INFO     {'Base': DecisionTreeRegressor(criterion='friedman_mse', max_leaf_nodes=127,
                      min_samples_split=3, random_state=1), 'col_sample': 0.5, 'learning_rate': 0.007, 'minibatch_frac': 0.7000000000000001, 'n_estimators': 10000000, 'verbose_eval': 100, 'random_state': 13}


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [4:02:16<00:00, 145.37s/trial, best loss: 2.380576009400627]

2021-07-28 11:39:44 INFO     ...done


In [33]:
best_params = space_eval(space, best)
best_params

{'Base': DecisionTreeRegressor(criterion='friedman_mse', max_leaf_nodes=63,
                       min_samples_split=3, random_state=1),
 'col_sample': 0.6000000000000001,
 'learning_rate': 0.009000000000000001,
 'minibatch_frac': 1.0}

## NGBoost

In [10]:
# Arrival

tree_learner = DecisionTreeRegressor(criterion = "friedman_mse",                 # “mse”, “friedman_mse”, “mae”, “poisson”
                                     min_samples_split = 3,             # The minimum number of samples required to split an internal node
                                     min_samples_leaf = 1,              # The minimum number of samples required to be at a leaf node
                                     min_weight_fraction_leaf = 0.0,    # The minimum weighted fraction of the sum total of weights required to be at a leaf node
                                     max_depth = None,                  # The maximum depth of the tree
                                     max_leaf_nodes = 127,              # Grow a tree with 'max_leaf_nodes' in best-first fashion
                                     splitter = "best",                 # The strategy used to choose the split at each node
                                     random_state = seed)


ngb_arrival = NGBRegressor(Dist = Normal,               # A distribution from ngboost.distns : Normal, LogNormal, Exponential...
                           Score = MLE,            # rule to compare probabilistic predictions P̂ to the observed data y, from ngboost.scores : LogScore, CRPScore...
                           Base = tree_learner,         # base learner to use in the boosting algorithm
                           natural_gradient = True,     # logical flag indicating whether the natural gradient should be used
                           verbose = True,
                           n_estimators = 10000000, 
                           learning_rate = 0.001,
                           minibatch_frac = 0.8,        # the percent subsample of rows to use in each boosting iteration
                           col_sample = 0.8,            
                           tol = 1e-5,                  # numerical tolerance to be used in optimization
                           random_state = 13)

ngb_arrival.fit(X_train_a, y_train_a,
                X_val = X_val_a,
                Y_val = y_val_a,
                sample_weight = None,                   # Weights of training data
                val_sample_weight = None,               # Weights of eval data
                train_loss_monitor = None,              # custom score or set of scores to track on the training set during training
                val_loss_monitor = None,                # custom score or set of scores to track on the validation set during training
                early_stopping_rounds = 2)

[iter 0] loss=3.8347 val_loss=3.8420 scale=2.0000 norm=18.2020
[iter 100] loss=3.6235 val_loss=3.6285 scale=2.0000 norm=15.1905
[iter 200] loss=3.4880 val_loss=3.4959 scale=2.0000 norm=12.7322
[iter 300] loss=3.3786 val_loss=3.3888 scale=2.0000 norm=10.7651
[iter 400] loss=3.2837 val_loss=3.2936 scale=2.0000 norm=9.2872
[iter 500] loss=3.1939 val_loss=3.2056 scale=2.0000 norm=8.0929
[iter 600] loss=3.1045 val_loss=3.1224 scale=2.0000 norm=7.0648
[iter 700] loss=3.0231 val_loss=3.0437 scale=2.0000 norm=6.3304
[iter 800] loss=2.9425 val_loss=2.9691 scale=2.0000 norm=5.7035
[iter 900] loss=2.8662 val_loss=2.8985 scale=2.0000 norm=5.2475
[iter 1000] loss=2.7938 val_loss=2.8321 scale=2.0000 norm=4.8883
[iter 1100] loss=2.7230 val_loss=2.7702 scale=2.0000 norm=4.5895
[iter 1200] loss=2.6553 val_loss=2.7129 scale=2.0000 norm=4.3377
[iter 1300] loss=2.5911 val_loss=2.6607 scale=2.0000 norm=4.1761
[iter 1400] loss=2.5303 val_loss=2.6138 scale=2.0000 norm=4.0313
[iter 1500] loss=2.4741 val_loss=

NGBRegressor(Base=DecisionTreeRegressor(criterion='friedman_mse',
                                        max_leaf_nodes=127, min_samples_split=3,
                                        random_state=1),
             col_sample=0.8, learning_rate=0.001, minibatch_frac=0.8,
             n_estimators=10000000,
             random_state=RandomState(MT19937) at 0x2E5B376CA40, tol=1e-05)

In [9]:
# Departure

tree_learner = DecisionTreeRegressor(criterion = "friedman_mse",                 # “mse”, “friedman_mse”, “mae”, “poisson”
                                     min_samples_split = 3,             # The minimum number of samples required to split an internal node
                                     min_samples_leaf = 1,              # The minimum number of samples required to be at a leaf node
                                     min_weight_fraction_leaf = 0.0,    # The minimum weighted fraction of the sum total of weights required to be at a leaf node
                                     max_depth = None,                  # The maximum depth of the tree
                                     max_leaf_nodes = 127,              # Grow a tree with 'max_leaf_nodes' in best-first fashion
                                     splitter = "best",                 # The strategy used to choose the split at each node
                                     random_state = seed)


ngb_departure = NGBRegressor(Dist = Normal,               # A distribution from ngboost.distns : Normal, LogNormal, Exponential...
                             Score = MLE,            # rule to compare probabilistic predictions P̂ to the observed data y, from ngboost.scores : LogScore, CRPScore...
                             Base = tree_learner,         # base learner to use in the boosting algorithm
                             natural_gradient = True,     # logical flag indicating whether the natural gradient should be used
                             verbose = True,
                             n_estimators = 10000000, 
                             learning_rate = 0.001,
                             minibatch_frac = 0.8,        # the percent subsample of rows to use in each boosting iteration
                             col_sample = 0.8,            
                             tol = 1e-5,                  # numerical tolerance to be used in optimization
                             random_state = 13)


ngb_departure.fit(X_train_d, y_train_d,
                X_val = X_val_d,
                Y_val = y_val_d,
                sample_weight = None,                   # Weights of training data
                val_sample_weight = None,               # Weights of eval data
                train_loss_monitor = None,              # custom score or set of scores to track on the training set during training
                val_loss_monitor = None,                # custom score or set of scores to track on the validation set during training
                early_stopping_rounds = 10)

[iter 0] loss=3.9479 val_loss=3.9350 scale=2.0000 norm=20.8821
[iter 100] loss=3.7300 val_loss=3.7299 scale=2.0000 norm=17.1325
[iter 200] loss=3.5941 val_loss=3.5947 scale=2.0000 norm=14.3145
[iter 300] loss=3.4794 val_loss=3.4836 scale=2.0000 norm=11.9403
[iter 400] loss=3.3748 val_loss=3.3821 scale=2.0000 norm=10.0322
[iter 500] loss=3.2743 val_loss=3.2871 scale=2.0000 norm=8.4558
[iter 600] loss=3.1806 val_loss=3.1965 scale=2.0000 norm=7.2549
[iter 700] loss=3.0898 val_loss=3.1093 scale=2.0000 norm=6.3017
[iter 800] loss=2.9978 val_loss=3.0256 scale=2.0000 norm=5.4791
[iter 900] loss=2.9094 val_loss=2.9450 scale=2.0000 norm=4.8926
[iter 1000] loss=2.8282 val_loss=2.8680 scale=2.0000 norm=4.4429
[iter 1100] loss=2.7422 val_loss=2.7947 scale=2.0000 norm=4.0576
[iter 1200] loss=2.6576 val_loss=2.7255 scale=2.0000 norm=3.7433
[iter 1300] loss=2.5787 val_loss=2.6611 scale=2.0000 norm=3.5249
[iter 1400] loss=2.4975 val_loss=2.6016 scale=2.0000 norm=3.3067
[iter 1500] loss=2.4243 val_loss

NGBRegressor(Base=DecisionTreeRegressor(criterion='friedman_mse',
                                        max_leaf_nodes=127, min_samples_split=3,
                                        random_state=1),
             col_sample=0.8, learning_rate=0.001, minibatch_frac=0.8,
             n_estimators=10000000,
             random_state=RandomState(MT19937) at 0x25D3D0C0840, tol=1e-05)

In [10]:
# Arrival

print("Train set RMSE : " , np.sqrt(mean_squared_error(y_train_a, ngb_arrival.pred_dist(X_train_a).loc)))
print("Test set RMSE : " , np.sqrt(mean_squared_error(y_test_a, ngb_arrival.pred_dist(X_test_a).loc)))

print("Train set R^2 : " , r2_score(y_train_a, ngb_arrival.pred_dist(X_train_a).loc))
print("Test set R^2 : " , r2_score(y_test_a, ngb_arrival.pred_dist(X_test_a).loc))

Train set RMSE :  1.668103108250137
Test set RMSE :  2.711712704987977
Train set R^2 :  0.9776415087323574
Test set R^2 :  0.9391631861609595


In [11]:
# Departure

print("Train set RMSE : " , np.sqrt(mean_squared_error(y_train_d, ngb_departure.pred_dist(X_train_d).loc)))
print("Test set RMSE : " , np.sqrt(mean_squared_error(y_test_d, ngb_departure.pred_dist(X_test_d).loc)))

print("Train set R^2 : " , r2_score(y_train_d, ngb_departure.pred_dist(X_train_d).loc))
print("Test set R^2 : " , r2_score(y_test_d, ngb_departure.pred_dist(X_test_d).loc))

Train set RMSE :  1.4508842206469068
Test set RMSE :  2.6205980459942544
Train set R^2 :  0.9864475680404998
Test set R^2 :  0.9536169941521409


In [11]:
y_a_pred = ngb_arrival.pred_dist(X_a)
predictions = pd.DataFrame(y_a_pred.loc, columns=['Predictions'])
predictions_sd = pd.DataFrame(y_a_pred.scale, columns=['Standard Deviation'])
predictions_upper = pd.DataFrame(y_a_pred.dist.interval(0.95)[1], columns=['95% Predictions_upper'])    # 95% prediction interval
predictions_lower = pd.DataFrame(y_a_pred.dist.interval(0.95)[0], columns=['95% Predictions_lower'])
Actual_AAR = pd.DataFrame({'Actual AAR':y_a})
Date = pd.date_range(start='1/1/2019', end='12/31/2019 23:00', freq = '1H')

prediction =  pd.concat([pd.DataFrame({'Date':Date}), Actual_AAR, predictions, predictions_sd, 
                         predictions_upper, predictions_lower], axis = 1)

In [13]:
prediction

,Date,Actual AAR,Predictions,Standard Deviation,95% Predictions_upper,95% Predictions_lower
0,2019-01-01 00:00:00,4.0,4.474189,2.913600,10.184741,-1.236363
1,2019-01-01 01:00:00,2.0,2.225787,3.097105,8.296000,-3.844426
2,2019-01-01 02:00:00,0.0,1.784956,3.161070,7.980540,-4.410629
3,2019-01-01 03:00:00,6.0,5.682507,3.049627,11.659667,-0.294652
4,2019-01-01 04:00:00,20.0,20.936671,2.103743,25.059932,16.813410
...,...,...,...,...,...,...
8755,2019-12-31 19:00:00,26.0,28.490084,2.584104,33.554834,23.425334
8756,2019-12-31 20:00:00,39.0,35.986559,3.638105,43.117115,28.856004
8757,2019-12-31 21:00:00,23.0,23.292020,2.240638,27.683590,18.900450
8758,2019-12-31 22:00:00,20.0,20.232461,2.245879,24.634304,15.830618


In [ ]:
def plot_result(prediction, start=0, end=10):   

    fig, ax = plt.subplots(figsize=(22, 10))
    plt.fill_between(prediction['Date'][start:end], prediction['95% Predictions_lower'][start:end],  prediction['95% Predictions_upper'][start:end], 
                     label = '95% Prediction Interval', color='gray', alpha=0.5)
    plt.plot(prediction['Date'][start:end], prediction['Predictions'][start:end], label = 'Predictions', lw=2)
    plt.scatter(prediction['Date'][start:end], prediction['Predictions'][start:end], lw=3)
    plt.scatter(prediction['Date'][start:end], prediction['Actual AAR'][start:end], label = 'Actual AAR', color='r', lw=3)

    ax.legend(fontsize = 15)
    #plt.title('Hourly Power Consumption Actual vs. Predicted Values with Prediction Intervals')
    plt.xlabel('Time')
    plt.ylabel('Arrivals per hour')
    plt.show()

In [ ]:
plot_result(prediction, start = 8230, end = 8300)

# NGBoost - example

## NGBoost

In [ ]:
X, Y = load_boston(True)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
# prediction
Y_preds = ngb.predict(X_test)

In [ ]:
# estimated distributional parameters at a set of point
Y_dists = ngb.pred_dist(X_test)

Y_dists[0:5].params    

# loc : mean
# scale : standard deviation

In [ ]:
# test Mean Squared Error
test_MSE = mean_squared_error(Y_preds, Y_test)
print('Test MSE', test_MSE)

# test Negative Log Likelihood
test_NLL = -Y_dists.logpdf(Y_test).mean()
print('Test NLL', test_NLL)

## Survival Regression

In [ ]:
import numpy as np
from ngboost import NGBSurvival
from ngboost.distns import LogNormal

# Distribution Estimator